# Even / Odd training
N2N (Noise2Noise) denoising del volúmen epfl1. Utiliza 2 volúmenes $V_1$ y $V_2$, donde $V_1$ es el volúmen formado por los slices Z pares "even" y $V_2$ es el volúmen formado por los slices Z impares "odd".

Todos los ficheros temporales se crean en `/nas/vruiz/cryoCARE/epfl1`.

## Creamos $V_1$ y $V_2$

In [ ]:
%run even_odd_split.ipynb

## Configuramos cryoCARE

In [ ]:
%%writefile /nas/vruiz/cryoCARE/epfl1/train_data_config__evenodd.json
{
    "even": ["even.mrc"],
    "odd": ["odd.mrc"],
    "mask": [""],
    "patch_shape": [32, 32, 32],
    "num_slices": 360,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./even_odd_data",
    "overwrite": "True"  
}

In [ ]:
%%writefile /nas/vruiz/cryoCARE/epfl1/train_config__evenodd.json
{
  "train_data": "./even_odd_data",
  "epochs": 50,
  "steps_per_epoch": 200,
  "batch_size": 16,
  "unet_kern_size": 3,
  "unet_n_depth": 3,
  "unet_n_first": 16,
  "learning_rate": 0.0004,
  "model_name": "model",
  "path": "./",
  "gpu_id": 1
}

In [ ]:
%%writefile /nas/vruiz/cryoCARE/epfl1/predict_config__evenodd.json
{
    "path": "./model.tar.gz",
    "even": ["epfl1_subset1_float.mrc"], 
    "odd": ["epfl1_subset1_float.mrc"],
    "n_tiles": [1,1,1],
    "output": "even_odd_denoised",
    "overwrite": "True",
    "gpu_id": 1
}

## Entrenamos

In [ ]:
%%bash
cd /nas/vruiz/cryoCARE/epfl1
pwd
source ~/envs/cryoCARE/bin/activate
cryoCARE_extract_train_data.py --conf train_data_config__evenodd.json
cryoCARE_train.py --conf train_config__evenodd.json

## Inferimos

In [ ]:
%%bash
cd /nas/vruiz/cryoCARE/epfl1
pwd
source ~/envs/cryoCARE/bin/activate
cryoCARE_predict.py --conf predict_config__evenodd.json || true

In [ ]:
import mrcfile
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
def read_MRC(path_file):
    return mrcfile.read(mrc_file_path)

In [ ]:
mrc_file_path = '/nas/vruiz/cryoCARE/epfl1/epfl1_subset1_float.mrc'
original_volume = read_MRC(mrc_file_path)

In [ ]:
original_volume.shape

In [ ]:
mrc_file_path = '/nas/vruiz/cryoCARE/epfl1/even_odd_denoised/epfl1_subset1_float.mrc'
denoised_volume = read_MRC(mrc_file_path)

In [ ]:
denoised_volume.shape

In [ ]:
# Choose a slice index in the middle of the volume for a good comparison
slice_idx = original_volume.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N Even/Odd Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()